In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import src.aformer_atac_cage as aformer
#import src.aformer_TF as aformer
from src.layers.layers import *
import src.metrics as metrics
from src.optimizers import *
import src.schedulers as schedulers
import src.utils as utils

import training_utils_atac as training_utils


from scipy import stats

2023-03-22 16:20:06.230115: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 16:20:06.404075: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-22 16:20:06.404108: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-22 16:20:07.280722: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-2')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    #options.experimental_threading.max_intra_op_parallelism = 1
    mixed_precision.set_global_policy('mixed_bfloat16')
    tf.config.optimizer.set_jit(True)
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 1
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS

2023-03-22 16:20:08.640841: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-22 16:20:08.640905: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-22 16:20:08.640929: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


2023-03-22 16:20:08.959695: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 16:20:08.984257: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:447] Started server with target: grpc://localhost:47304
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: node-2


INFO:tensorflow:Initializing the TPU system: node-2


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
g = tf.random.Generator.from_seed(1)
with strategy.scope():
    data_tr,data_val,data_val_ho= training_utils.return_distributed_iterators("gs://picard-testing-176520/genformer_atac_pretrain/393k/genformer_atac_pretrain_globalacc_conv_rpgc",
                                                                            "gs://picard-testing-176520/genformer_atac_pretrain/393k/genformer_atac_pretrain_globalacc_conv_rpgc",
                                                                           GLOBAL_BATCH_SIZE,
                                                                           393216,
                                                                           10,
                                                                            98304,
                                                                           3072,
                                                                           1088,
                                                                           128,
                                                                           8,
                                                                           50,
                                                                           strategy,
                                                                           options,
                                                                            0.10,
                                                                              1024,
                                                                              True,
                                                                              g)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [4]:
with strategy.scope():
    #inits = training_utils.get_initializers_enformer_performer("gs://picard-testing-176520/enformer_performer_pretrain_atac_mean/models/enformer_performer_pretrain_atac_mean_EP_full_dataset__196k_load_init-True_freeze-False_LR1-1e-06_LR2-0.0001_T-4_F-1536_D-0.4_2023-02-24_16:27:50/iteration_16",
    #                                                           4,
     #                                                          stable_variant=False)
    inits = training_utils.get_initializers_enformer_conv("gs://picard-testing-176520/sonnet_weights/sonnet_weights",
                                                          True,
                                                          6)
    #inits = training_utils.get_initializers_enformer_performer("gs://picard-testing-176520/genformer_atac_pretrain/models/aformer_baseline_genformer_atac_196k_glob_acc-False_load-True_frz-False_LR1-7.5e-06_LR2-0.00025_T-9_D-0.05_2023-03-18_01:43:51/iteration_40",
    #                                                          9,
    #                                                           stable_variant=False)
                                                          
    model = aformer.aformer(kernel_transformation = 'relu_kernel_transformation',
                             dropout_rate = 0.05,
                             pointwise_dropout_rate=0.1,
                             input_length = 393216,
                             output_length = 3072,
                             final_output_length = 896,
                             num_heads = 8,
                             numerical_stabilizer = 0.001,
                             nb_random_features = 256,
                             num_transformer_layers = 4,
                             norm=True,
                             BN_momentum=0.90,
                             max_seq_length = 3072,
                             use_rot_emb = True,
                             use_mask_pos = False, 
                             normalize = True,
                             seed = 3,
                             load_init=False,
                             stable_variant=False,
                             inits=inits,
                             freeze_conv_layers=False,
                             learnable_PE=True,
                             filter_list_seq=[512,640,768,896,1024,1152],
                             inits_type='enformer_conv')

In [5]:
with strategy.scope():
    optimizer1 = tfa.optimizers.AdaBelief(
        learning_rate= 1.0e-06,
        epsilon= 1.0e-14,
        weight_decay= 1.0e-08,
        rectify=True
    )
    optimizer2 = tfa.optimizers.AdaBelief(
        learning_rate= 1.0e-04,
        epsilon= 1.0e-14,
        weight_decay= 1.0e-08,
        rectify=True
    )
    
    optimizers_in=optimizer1,optimizer2
    

In [6]:
with strategy.scope():
    metric_dict = {}
    train_step, val_step,build_step, metric_dict = training_utils.return_train_val_functions(model,
                                                                                15,
                                                                                10,
                                                                                optimizers_in,
                                                                                  strategy,
                                                                                  metric_dict,
                                                                                  GLOBAL_BATCH_SIZE,
                                                                                10.0)

In [7]:
from scipy.stats.stats import pearsonr  
with strategy.scope():
    def sum_log(x):
        return np.log10(1.0 + np.nansum(x))
    
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(1, 10):
        start = time.time()
        if epoch_i == 1:
            build_step(data_val)
            total_params = 0
            for k in model.trainable_variables:
                var = k.values[0]
                total_params += tf.size(var)
            print('total params: ' + str(total_params))


        train_step(data_tr)
        #train_step(data_tr)

        print('hg_train_loss: ' + str(metric_dict['train_loss'].result().numpy()))
        
        val_step(data_val)
        #val_step(data_val)

        val_losses.append(metric_dict['val_loss'].result().numpy())
        print('hg_val_loss: ' + str(metric_dict['val_loss'].result().numpy()))
        #print('hg_val_loss_CAGE: ' + str(metric_dict['val_loss_CAGE'].result().numpy()))
        #print('hg_val_loss_ATAC: ' + str(metric_dict['val_loss_ATAC'].result().numpy()))
        #print('hg_val_cage_pearson: ' + str(metric_dict['CAGE_PearsonR'].result()['PearsonR'].numpy()))
        #print('hg_val_cage_R2: ' + str(metric_dict['CAGE_R2'].result()['R2'].numpy()))
        
        print('hg_val_atac_pearson: ' + str(metric_dict['ATAC_PearsonR'].result()['PearsonR'].numpy()))
        print('hg_val_atac_R2: ' + str(metric_dict['ATAC_R2'].result()['R2'].numpy()))
        
        
        #print('hg_val_atac_baseline_pearson: ' + str(metric_dict['ATAC_PearsonR_baseline'].result()['PearsonR'].numpy()))
        #print('hg_val_atac_baseline_R2: ' + str(metric_dict['ATAC_R2_baseline'].result()['R2'].numpy()))
        """
        val_step_TSS(data_val_TSS)
        #val_step_TSS(data_val_TSS)
        
        y_trues = metric_dict['corr_stats'].result()['y_trues'].numpy()
        y_preds = metric_dict['corr_stats'].result()['y_preds'].numpy()
        cell_types = metric_dict['corr_stats'].result()['cell_types'].numpy()
        gene_map = metric_dict['corr_stats'].result()['gene_map'].numpy()


        figures, corrs_overall = training_utils.make_plots(y_trues,
                                           y_preds, 
                                           cell_types, 
                                           gene_map,50)
        
        cell_spec,gene_spec,cell_spec_sp,gene_spec_sp = corrs_overall
        
        print('cell spec corr: ' + str(cell_spec))
        print('gene_spec_corr:' + str(gene_spec))
        print('cell spec corr_sp: ' + str(cell_spec_sp))
        print('gene_spec_corr_sp:' + str(gene_spec_sp))
        """
        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('duration(mins): ' + str(duration))
        print('patience counter at: ' + str(patience_counter))
        
        for key, item in metric_dict.items():
            item.reset_state()
        

/opt/conda/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


total params: tf.Tensor(56473298, shape=(), dtype=int32)
hg_train_loss: 1.2085774
hg_val_loss: 2.2721503
hg_val_atac_pearson: 0.009867602
hg_val_atac_R2: -0.03447008
completed epoch 1
duration(mins): 5.3339367508888245
patience counter at: 0
hg_train_loss: 0.6265086
hg_val_loss: 2.3077803
hg_val_atac_pearson: -0.05564437
hg_val_atac_R2: -0.04941213
completed epoch 2
duration(mins): 4.471023921171824
patience counter at: 0
hg_train_loss: -0.30015162
hg_val_loss: 1.7875569
hg_val_atac_pearson: -0.047405142
hg_val_atac_R2: -0.057927847
completed epoch 3
duration(mins): 4.513658805688222
patience counter at: 0
hg_train_loss: -1.5304525
hg_val_loss: 1.1633997
hg_val_atac_pearson: 0.023728712
hg_val_atac_R2: -0.04131615
completed epoch 4
duration(mins): 4.562556711832682
patience counter at: 0
hg_train_loss: -1.7878797
hg_val_loss: 0.20521978
hg_val_atac_pearson: 0.0361176
hg_val_atac_R2: -0.021263361
completed epoch 5
duration(mins): 4.53185107310613
patience counter at: 0
hg_train_loss: -1

2023-03-22 16:48:42.351228: W ./tensorflow/core/distributed_runtime/eager/destroy_tensor_handle_node.h:58] Ignoring an error encountered when deleting remote tensors handles: INVALID_ARGUMENT: Unable to find the relevant tensor remote_handle: Op ID: 20677, Output num: 1
Additional GRPC error information from remote target /job:worker/replica:0/task:0:
:{"created":"@1679503722.347823235","description":"Error received from peer ipv4:10.93.119.234:8470","file":"external/com_github_grpc_grpc/src/core/lib/surface/call.cc","file_line":1056,"grpc_message":"Unable to find the relevant tensor remote_handle: Op ID: 20677, Output num: 1","grpc_status":3}


FailedPreconditionError: SharedMemoryAddress expired.
	 [[{{node RemoteCall}}]]
Additional GRPC error information from remote target /job:worker/replica:0/task:0:
:{"created":"@1679503722.346955124","description":"Error received from peer ipv4:10.93.119.234:8470","file":"external/com_github_grpc_grpc/src/core/lib/surface/call.cc","file_line":1056,"grpc_message":"SharedMemoryAddress expired.\n\t [[{{node RemoteCall}}]]","grpc_status":9}

In [ ]:
with strategy.scope():
    g = tf.random.Generator.from_seed(1)
    

In [3]:
checkpoint_path="gs://picard-testing-176520/genformer_atac_pretrain/models/aformer_baseline_genformer_atac_196k_glob_acc-True_load-True_frz-False_LR1-5e-06_LR2-0.0001_T-7_D-0.05_2023-03-15_03:26:43/iteration_60"

inside_checkpoint=tf.train.list_variables(tf.train.latest_checkpoint(checkpoint_path))
reader = tf.train.load_checkpoint(checkpoint_path)

In [4]:
inside_checkpoint

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/batch_norm/beta/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/batch_norm/gamma/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/batch_norm/moving_mean/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/batch_norm/moving_variance/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-1/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-1/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [5, 768, 768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-1/_layer/layer_with_weights-0/batch_norm/beta/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_wit

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


ValueError: in user code:

    File "/home/jupyter/dev/aformer/training_utils_atac.py", line 828, in None  *
        g),
    File "/home/jupyter/dev/aformer/training_utils_atac.py", line 631, in deserialize_tr  *
        seq_mask = tf.expand_dims(tf.reshape(tf.tile(seq_mask, [1,tiling_req_seq]),[-1]),axis=1)

    ValueError: Shape must be rank 1 but is rank 2 for '{{node cond_10/Tile}} = Tile[T=DT_FLOAT, Tmultiples=DT_INT32](cond_10/sub, cond_10/Tile/multiples)' with input shapes: [1536], [2].
